<a href="https://colab.research.google.com/github/keirwilliamsxyz/keirxyz/blob/main/Real_ESRGAN_Sber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Установка


Веса для x2, x4, x8 моделей можно скачать по этой ссылке:
https://drive.google.com/drive/folders/16PlVKhTNkSyWFx52RPb2hXPIQveNGbxS

In [ ]:
!git clone https://github.com/boomb0om/Real-ESRGAN-colab
%cd Real-ESRGAN-colab
!pip install -r requirements.txt
# download model weights
# x2 
!gdown https://drive.google.com/uc?id=1pG2S3sYvSaO0V0B8QPOl1RapPHpUGOaV -O weights/RealESRGAN_x2.pth
# x4
!gdown https://drive.google.com/uc?id=1SGHdZAln4en65_NQeQY9UjchtkEF9f5F -O weights/RealESRGAN_x4.pth
# x8
!gdown https://drive.google.com/uc?id=1mT9ewx86PSrc43b-ax47l1E2UzR7Ln4j -O weights/RealESRGAN_x8.pth

# Загрузка изображений/архивов


In [ ]:
import os
from google.colab import files
import shutil

upload_folder = 'inputs'
result_folder = 'results'

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

# upload files
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

# Запуск модели

In [ ]:
# Создание модели
from realesrgan import RealESRGAN
from PIL import Image
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model = RealESRGAN(device, scale=4)
model.load_weights('weights/RealESRGAN_x4.pth')

Улучшение одного изображения:

In [ ]:
path_to_image = 'inputs/lr_image.png'
result_image_path = 'results/sr_image.png'

image = Image.open(path_to_image).convert('RGB')
sr_image = model.predict(np.array(image))

sr_image.save(result_image_path)
sr_image

Улучшение фотографий в архиве:

In [ ]:
path_to_tar = 'inputs/test_upscale.tar.gz'
result_tar_path = 'results/test_upscale_x4.tar.gz'

In [ ]:
#@title Обработка фото в архиве

from io import BytesIO
import io
import tarfile

IMAGE_FORMATS = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

def image_to_tar_format(img, image_name):
  buff = BytesIO()
  if '.png' in image_name.lower():
    img = img.convert('RGBA')
    img.save(buff, format='PNG')
  else:
    img.save(buff, format='JPEG')
  buff.seek(0)
  fp = io.BufferedReader(buff)
  img_tar_info = tarfile.TarInfo(name=image_name)
  img_tar_info.size = len(buff.getvalue())
  return img_tar_info, fp

processing_tar = tarfile.open(path_to_tar, mode='r')
save_tar = tarfile.open(result_tar_path, 'w')

for c, member in enumerate(processing_tar):
  print(f'{c}, processing {member.name}')

  if not member.name.endswith(IMAGE_FORMATS):
    continue

  try:
    img_bytes = BytesIO(processing_tar.extractfile(member.name).read())
    img_lr = Image.open(img_bytes, mode='r').convert('RGB')
  except Exception as err:
    print(f'Не получилось открыть файл {member.name}, пропускаю')
    continue

  img_sr = model.predict(np.array(img_lr))

  # adding to save_tar
  img_tar_info, fp = image_to_tar_format(img_sr, member.name)
  save_tar.addfile(img_tar_info, fp)

processing_tar.close()
save_tar.close()

print(f'Finished! Archive saved to {result_tar_path}')